In [1]:
# For Google Colab
# Install folium package.
!pip install folium

In [ ]:
# For Google Colab
# Mount your Google drive and copy all files from "AI_HW2" directory
# in your Google drive to current directory.
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r ./gdrive/MyDrive/0616076_hw2/* .

Mounted at /content/gdrive


In [5]:
# Don't change this part.
# For load graph information and show map
import folium
import pickle
def load_path_graph(path):
    with open('graph.pkl', 'rb') as f:
        graph = pickle.load(f)

    node_pairs = list(zip(path[:-1], path[1:]))
    lines = []
    for edge in graph:
        if (edge['u'], edge['v']) in node_pairs or  (edge['v'], edge['u']) in node_pairs:
            lines.append(edge['geometry'])
    return lines

In [4]:
# Part 1~4 and 6
# You can write your code in this file directly.
# Or you can wirte in new .py files and import it.
# ex: from astar import astar
sample_text = open("gdrive/MyDrive/0616076_hw2/edges.csv")
txt = sample_text.readlines()
start_node=[]
end_node=[]
cost = []
for i in range(1,len(txt)):
  a, b, c, d = txt[i].split(',')
  start_node.append(a)
  end_node.append(b)
  cost.append(c)
#for i,j,k in zip(start_node, end_node, cost):
#  print(i,"->",j,"=",k)


temp1 = set(start_node)
temp2 = set(end_node)
total_states = temp1.union(temp2)
print("Total states= ",total_states)
total_states = list(total_states)

values = dict()
for i in total_states:
  values[i] = []
for i in total_states:
  for j in range(0, len(start_node)):
    if i == start_node[j]:
      values[i].append(end_node[j])
print(values)

visited = [] # List to keep track of visited nodes.
Bvisited = []
queue = []     #Initialize a queue


def bfs(start, end):
  start = str(start)
  end = str(end)
  visited.append(start)
  Bvisited.append(-1)
  queue.append(start)
  bfs_path = []
  bfs_dist = 0
  bfs_visited = []
  
  while queue:
    s = queue.pop(0) 
    bfs_visited.append(s) 
    #print(bfs_path)
    if s == end:
      break
    for neighbour in values[s]:
      #print(neighbour)
      if neighbour not in visited:
        visited.append(neighbour)
        Bvisited.append(s)
        queue.append(neighbour)
  #print("visited:", visited)  
  #print("Bvisited:", Bvisited)
  #print("bfs_visited:", bfs_visited)
  t = 1
  for i in range(len(bfs_visited)-1, -1, -1):
    #print(bfs_visited[i])
    for j in range(len(visited)-1, -1, -1):
      if t == 1 and bfs_visited[i] == visited[j]:
        bfs_path.append(int(Bvisited[j]))
        t = Bvisited[j]
      elif bfs_visited[i] == visited[j] and bfs_visited[i] == t :
        bfs_path.append(int(Bvisited[j]))
        t = Bvisited[j]
  bfs_path.reverse()
  bfs_path.append(int(end))    
  bfs_path.pop(0)
  for j in range(0, len(bfs_path)-1):
    for i in range(1,len(txt)-1):
      if int(start_node[i]) == int(bfs_path[j]) and int(end_node[i]) == int(bfs_path[j+1]):
        bfs_dist += float(cost[i])
        
        #print(i, j, bfs_dist)
  return bfs_path, bfs_dist, len(bfs_visited)

dvisited = [] # List to keep track of visited nodes.
dBvisited = []
dstack = []     
def dfs(start, end):
  start = str(start)
  end = str(end)
  dvisited.append(start)
  dBvisited.append(-1)
  dstack.append(start)
  dfs_path = []
  dfs_dist = 0
  dfs_visited = []
  while dstack:
    s = dstack.pop() 
    dfs_visited.append(s) 
    #print(bfs_path)
    if s == end:
      break
    for neighbour in values[s]:
      #print(neighbour)
      if neighbour not in dvisited:
        dvisited.append(neighbour)
        dBvisited.append(s)
        dstack.append(neighbour)
  #print("visited:", visited)  
  #print("Bvisited:", Bvisited)
  #print("bfs_visited:", bfs_visited)
  t = 1
  for i in range(len(dfs_visited)-1, -1, -1):
    #print(bfs_visited[i])
    for j in range(len(dvisited)-1, -1, -1):
      if t == 1 and dfs_visited[i] == dvisited[j]:       
        dfs_path.append(int(dBvisited[j]))
        t = dBvisited[j]       
      elif dfs_visited[i] == dvisited[j] and dfs_visited[i] == t :       
        dfs_path.append(int(dBvisited[j]))
        t = dBvisited[j]   

  #print("bfs_path:", bfs_path)
  dfs_path.reverse()
  dfs_path.append(end)    
  dfs_path.pop(0)
  for j in range(0, len(dfs_path)-1):
    for i in range(1,len(txt)-1):
      if int(start_node[i]) == int(dfs_path[j]) and int(end_node[i]) == int(dfs_path[j+1]):
        dfs_dist += float(cost[i])  

        
        #print(i, j, bfs_dist)
  return dfs_path, dfs_dist, len(dfs_visited)


import queue as Q
graph = {}
for i in range(1,len(txt)-1):  #len    
  graph[start_node[i], end_node[i]] = cost[i]

def ucs(start, end):
  start = str(start)
  end = str(end)
  usc_path = []
  usc_dist = 0
  usc_visited = []
  queue = Q.PriorityQueue()
  queue.put((0, [start]))
  while not queue.empty():
    node = queue.get()
    current = node[1][len(node[1]) - 1]
    
    if end in str(node[1]):
      for i in range(0, len(node[1][:])):
        if node[1][:][i] not in usc_visited:
          usc_visited.append(node[1][:][i])
      break
        
    costv = node[0]   
    for i in range(0, len(node[1][:])):
      if node[1][:][i] not in usc_visited:
        usc_visited.append(node[1][:][i])
    for neighbor in values[current]:
      temp = node[1][:]
      temp.append(neighbor)
      queue.put((float(costv) + float(graph[current, neighbor]), temp))

  for i in range(0, len(node[1])):
    usc_path.append(int(node[1][i]))    #int
  usc_dist = node[0]
  return usc_path, usc_dist, len(usc_visited)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Part 5
# Change start ID and end ID.
start = 2270143902
end =  1079387396

In [ ]:
# Don't change this part.
# Show the result of BFS
bfs_path, bfs_dist, bfs_visited = bfs(start,end)
print(f'The number of nodes in the path found by BFS: {len(bfs_path)}')
print(f'Total distance of path found by BFS: {bfs_dist} m')
print(f'The number of visited nodes in BFS: {bfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(bfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='bfs', weight=4, color='blue'))
fmap

The number of nodes in the path found by BFS: 88
Total distance of path found by BFS: 4978.8820000000005 m
The number of visited nodes in BFS: 4274



In [ ]:
# Don't change this part.
# Show the result of DFS
dfs_path, dfs_dist, dfs_visited = dfs(start, end)
print(f'The number of nodes in the path found by DFS: {len(dfs_path)}')
print(f'Total distance of path found by DFS: {dfs_dist} m')
print(f'The number of visited nodes in DFS: {dfs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(dfs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='dfs', weight=4, color='green'))
fmap

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Don't change this part.
# Show the result of UCS
ucs_path, ucs_dist, ucs_visited = ucs(start, end)
print(f'The number of nodes in the path found by UCS: {len(ucs_path)}')
print(f'Total distance of path found by UCS: {ucs_dist} m')
print(f'The number of visited nodes in UCS: {ucs_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(ucs_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='ucs', weight=4, color='violet'))
fmap

In [ ]:
# Don't change this part.
# Show the result of A* search
astar_path, astar_dist, astar_visited = astar(start, end)
print(f'The number of nodes in the path found by A* search: {len(astar_path)}')
print(f'Total distance of path found by A* search: {astar_dist} m')
print(f'The number of visited nodes in A* search: {astar_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(astar_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap

NameError: ignored

In [ ]:
# Part 6 (Bonus)
# Don't change this part.
# Show the shortest time result of A* search
time_path, time, time_visited = astar_time(start, end)
print(f'The number of nodes in the path found by A* search: {len(time_path)}')
print(f'Total second of path found by A* search: {time} s')
print(f'The number of visited nodes in A* search: {time_visited}\n')

fmap = folium.Map(location=(24.806383132251874, 120.97685775516189), zoom_start=13)
for line in load_path_graph(time_path):
    fmap.add_child(folium.PolyLine(locations=line, tooltip='astar', weight=4, color='red'))
fmap

The number of nodes in the path found by A* search: 83
Total second of path found by A* search: 320.3284234929006 s
The number of visited nodes in A* search: 359



In [ ]:
# For Google Colab
# Remember to execute this line once you've modified any .py file
# Save the .py code you have modified to your Google Drive
!cp ./*.py ./gdrive/MyDrive/AI_HW2/

cp: cannot stat './*.py': No such file or directory
